<a href="https://colab.research.google.com/github/kavyajeetbora/monitoring_water_surface_area/blob/master/notebooks/monitoring_inland_water_area.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monitoring Inland Surface Water Area

In [3]:
import geemap
import ee
import shapely

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Area of Interest

In [8]:
y1,x1 = 27.5029,92.0958
y2,x2 = 27.5157,92.1097

bbox = (x1, y1, x2, y2)

polygon = shapely.geometry.box(*bbox, ccw=True)
geojson = eval(shapely.to_geojson(polygon))

geometry = ee.Geometry(geojson)

In [9]:
geometry

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              92.1097,
              27.5029
            ],
            [
              92.1097,
              27.5157
            ],
            [
              92.0958,
              27.5157
            ],
            [
              92.0958,
              27.5029
            ],
            [
              92.1097,
              27.5029
            ]
          ]
        ]
      }
    }
  }
})

## Download images from google earth

In [22]:
def getImageCollection(image_id, dates, geometry, cloud_cover):
    d1,d2 = dates
    ic = ee.ImageCollection(image_id)\
    .filter(ee.Filter.date(d1,d2))\
    .filter(ee.Filter.bounds(geometry))\
    .filter(ee.Filter.lt('CLOUD_COVER', cloud_cover))

    return ic

def preprocess_landsat(image:ee.image.Image) -> ee.image.Image:
    '''
    This function will choose only the required bands
    also rescale the reflectance value to original
    and calculate the indices: NDVI
    '''

    ## Select and rescale
    bands = image.select("SR_*").multiply(2.75e-05).add(-2)

    ## Calculate the NDWI
    ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('ndwi')

    return ndwi.copyProperties(image, image.propertyNames())

In [21]:
cloud_cover = 5
dates = ("1999-01-01", "2025-01-01")
image_ids = ["LANDSAT/LE07/C02/T1_L2", "LANDSAT/LC08/C02/T1_L2","LANDSAT/LC09/C02/T1_L2"]

lc = None

for i, image_id in enumerate(image_ids):
    lc_images = getImageCollection(
        image_id = image_id,
        dates = dates,
        geometry = geometry,
        cloud_cover = cloud_cover
    )
    if i == 0:
        lc = lc_images
    else:
        lc = lc.merge(lc_images)

lc.size()

In [ ]:
lc.map(preprocess_landsat)